In [ ]:
import numpy as np
from uav_boxes import Village
from seeding_utils import point_near_regions, vis_reg, point_in_regions
from scipy.sparse import lil_matrix
import numpy as np
from tqdm import tqdm
from region_generation import generate_regions_multi_threading

seed = 46

world = Village()
village_side = 39
village_height = 10
world.build(village_height=village_height, village_side=village_side, building_every=5, density=0.15, seed=seed)
#vgraph_builder = world.to_drake_plant()
print(len(world.obstacles))

In [ ]:
import pickle
from pydrake.all import HPolyhedron, VPolytope


regions = []
with open('experiment_village_greedy_39_10_09_27_04_59_24___46_500_0.050_0.200/data/it_6.pkl', 'rb') as f:
    data= pickle.load(f)

for rga, rgb in zip(data['ra'], data['rb']):
    for ra, rb in zip(rga, rgb):
        regions.append(HPolyhedron(ra, rb))

In [ ]:
intersections = []

for i in range(len(regions)):
    for j in range(i+1, len(regions)):
        if regions[i].IntersectsWith(regions[j]):
            intersections.append(regions[i].Intersection(regions[j]))
print(intersections[0])
print(len(intersections))

# keep things clean, as simple as possible, don´t focus on speed yet
# check for redundancy = yes
# And reduce inequalities
# use networkx A star
# use networkx's existing methods to ask it for an adjacency matrix, for A star, put it into this graph form FIRST

# Plotting Utils

In [ ]:
import os
import mcubes
from meshcat.geometry import TriangularMeshGeometry, MeshLambertMaterial
from matplotlib.colors import to_hex as _to_hex

to_hex = lambda rgb: '0x' + _to_hex(rgb)[1:]
def col_hand(q1,q2,q3):
    p =np.array([q1,q2,q3])
    # if np.any(world.L_dom>p):
    #     return 1
    # if np.any(world.U_dom<p):
    #     return 1
    return 1.0*world.col_handle(np.array([q1,q2,q3]))

def col_hand_neg(q1,q2,q3):
    p =np.array([q1,q2,q3])
    if np.any(world.L_dom>p):
        return 0
    if np.any(world.U_dom<p):
        return 0
    return 1- 1.0*world.col_handle(p)

def plot_collision_constraint(N = 50, q_min = np.array(world.L_dom)-5, q_max= np.array(world.U_dom)+5):
    if f"col_cons{N}.pkl" in os.listdir('tmp'):
        with open(f"tmp/col_cons{N}.pkl", 'rb') as f:
            d = pickle.load(f)
            vertices = d['vertices']
            triangles = d['triangles']
    else:  
        vertices, triangles = mcubes.marching_cubes_func(
        tuple(
                q_min), tuple(
                q_max), N, N, N, col_hand, 0.5)
        with open(f"tmp/col_cons{N}.pkl", 'wb') as f:
                d = {'vertices': vertices, 'triangles': triangles}
                pickle.dump(d, f)

    #tri_drake = [SurfaceTriangle(*t) for t in triangles]

    #vertices += _offset_meshcat_2.reshape(-1,3)
    obj = world['col']
    color = to_hex((1,0,0))
    material = MeshLambertMaterial(color=color, opacity=0.4,  wireframe=False)
    obj.set_object(TriangularMeshGeometry(vertices, triangles),
                                   material)

def plot_collision_constraint_neg(N = 50, q_min = np.array(world.L_dom)-5, q_max= np.array(world.U_dom)+5):
    if f"col_cons{N}_n.pkl" in os.listdir('tmp'):
        with open(f"tmp/col_cons{N}_n.pkl", 'rb') as f:
            d = pickle.load(f)
            vertices = d['vertices']
            triangles = d['triangles']
    else:  
        vertices, triangles = mcubes.marching_cubes_func(
        tuple(
                q_min), tuple(
                q_max), N, N, N, col_hand_neg, 0.5)
        with open(f"tmp/col_cons{N}_n.pkl", 'wb') as f:
                d = {'vertices': vertices, 'triangles': triangles}
                pickle.dump(d, f)

    #tri_drake = [SurfaceTriangle(*t) for t in triangles]

    #vertices += _offset_meshcat_2.reshape(-1,3)
    obj = world['col_n']
    color = to_hex((0,1,0))
    material = MeshLambertMaterial(color=color, opacity=0.3,  wireframe=False)
    obj.set_object(TriangularMeshGeometry(vertices, triangles),
                                   material)
from meshcat.geometry import Cylinder
import meshcat.transformations as tf
from meshcat.geometry import MeshLambertMaterial

def compute_rotation_matrix(a, b):
    # Normalize the points
    a = a / np.linalg.norm(a)
    b = b / np.linalg.norm(b)
    
    # Calculate the rotation axis
    rotation_axis = np.cross(a, b)
    rotation_axis /= np.linalg.norm(rotation_axis)
    
    # Calculate the rotation angle
    dot_product = np.dot(a, b)
    rotation_angle = np.arccos(np.clip(dot_product, -1.0, 1.0))
    
    # Construct the rotation matrix using Rodrigues' formula
    skew_matrix = np.array([[0, -rotation_axis[2], rotation_axis[1]],
                            [rotation_axis[2], 0, -rotation_axis[0]],
                            [-rotation_axis[1], rotation_axis[0], 0]])
    rotation_matrix = np.eye(3) + np.sin(rotation_angle) * skew_matrix + (1 - np.cos(rotation_angle)) * np.dot(skew_matrix, skew_matrix)
    
    return rotation_matrix

def plot_edge(pt1, pt2, name, color, opacity, size = 0.01):
    
    material = MeshLambertMaterial(color=to_hex(color), opacity=opacity)
    world[name].set_object(
                        Cylinder(np.linalg.norm(pt1-pt2), size),
                        material)
    
    dir = pt2-pt1
    rot = compute_rotation_matrix(np.array([0,1,0]), dir )
    #print(rot)
    offs = rot@np.array([0,np.linalg.norm(pt1-pt2)/2,0])
    tfmat = np.zeros((4,4))
    tfmat[:3,:3] = rot
    tfmat[:3,-1] = offs + pt1.squeeze()
    tfmat[3,3] = 1
    world[name].set_transform(tfmat)
    
def plot_edges(edges, name, color, opacity  = 1, size = 0.01):
    for i, e in enumerate(edges):
         plot_edge(e[0], e[1], name + f"/e_{i}", color, opacity, size=size)

#plot_edge(np.zeros(3), 5*np.ones(3), 'test', (1,0,0), 1, 0.8)

In [ ]:
world.plot_regions(regions, opacity=0.6)
from utils import plot_regions
 #this is for debugging the meldis meshcat -> activate kproximity to see the world
plot_regions(world.meshcat_drake, regions)

# Dummy Pathplanning and plotting of the resulting trajectory

In [ ]:
from dijkstraspp import DijkstraSPPsolver

def conv_dummy(q):
    return q

dspp = DijkstraSPPsolver(regions, conv_dummy)

In [ ]:
from meshcat.geometry import Sphere
point_in_regions(np.array([0,0, 0.5]), regions)
point_in_regions(np.array([38,38, 6.5]), regions)
world['start'].set_object(Sphere(0.6), MeshLambertMaterial(color=to_hex([0,1,0]), opacity=0.5))
world['start'].set_transform(tf.translation_matrix(np.array([0,0, 0.5])))
world['goal'].set_object(Sphere(0.6), MeshLambertMaterial(color=to_hex([0,1,0]), opacity=0.5))
world['goal'].set_transform(tf.translation_matrix(np.array([38,38, 6.5])))

In [ ]:
wps, dist = dspp.solve(np.array([0,0, 0.5]), np.array([38,38, 6.5]), refine_path=True)


In [ ]:
edges = []
for idx, wp in enumerate(wps[:-1]):
    edges.append([wp, wps[idx+1]])

plot_edges(edges, 'traj', (1,0,1), 1, 0.1 )

In [ ]:
from meshcat.transformations import translation_matrix, rotation_matrix
sign_size = [20.4, 20.2]  # Width and height of the sign
sign_position = [-20.0, 0.0, 10.0]  # X, Y, Z position of the sign
sign_color = [0.7, 0.7, 0.7]  # RGB color for the sign
from meshcat.geometry import MeshLambertMaterial, ImageTexture, Box
from PIL import Image
import meshcat
# Add the sign as a rectangle

# Load an image to put on the sign
image_path = "village_sign.png"

# Create a textured material for the sign
texture = ImageTexture(image = meshcat.geometry.PngImage.from_file(image_path))
textured_material = MeshLambertMaterial(map=texture)
sign = Box((sign_size[0], sign_size[1], 0.1))
world["sign"].set_object(sign, textured_material)
transl = translation_matrix(sign_position)
rot = rotation_matrix(np.pi/2, [1,0,0])
rot = rot@rotation_matrix(np.pi/2, [0,1,0])
rot[:,3] = transl[:,3]
world["sign"].set_transform(rot)



In [ ]:
# this saves the result to a html which you can share
# file = world.static_html()
# with open('Village.html', 'w') as f:
#     f.write(file)

In [ ]:
# if f"col_cons{100}_n.pkl" in os.listdir('tmp'):
#     with open(f"tmp/col_cons{100}_n.pkl", 'rb') as f:
#         d = pickle.load(f)
#         vertices = d['vertices']
#         triangles = d['triangles']

# output_file_path = "output_mesh.obj"

# # Write the mesh data to the .obj file
# with open(output_file_path, 'w') as obj_file:
#     obj_file.write("# Triangle Mesh\n")
    
#     # Write vertex coordinates
#     for vertex in vertices:
#         vertex = vertex
#         obj_file.write(f"v {' '.join(map(str, vertex))}\n")
    
#     # Write triangle definitions
#     for triangle in triangles:
#         # Add 1 to vertex indices because .obj files use 1-based indexing
#         obj_file.write(f"f {' '.join(map(lambda x: str(x + 1), triangle))}\n")

# print(f"Mesh saved to {output_file_path}")

# Generate new Regions (for a smaller example, see village side length)

In [ ]:
import numpy as np
from uav_boxes import Village
from seeding_utils import point_near_regions, vis_reg, point_in_regions
from scipy.sparse import lil_matrix
import numpy as np
from tqdm import tqdm
from region_generation import generate_regions_multi_threading
from utils import plot_regions

import numpy as np
from uav_boxes import Village
from seeding_utils import point_near_regions, vis_reg, point_in_regions
from scipy.sparse import lil_matrix
import numpy as np
from tqdm import tqdm
from region_generation import generate_regions_multi_threading


seed = 1
eps = 0.3 #this is 1- coverage -> if you want 90 coverage then set eps to 0.1
approach = 1 #leave this at 1, you will need gurobi for this
N = 100
ap_names = ['redu', 'greedy', 'nx', 'greedy_edge_CC', 'greedy_cvx_hull_fill']


world = Village()
village_side = 14 #change village side here
village_height = 10
world.build(village_height=village_height, village_side=village_side, building_every=5, density=0.15, seed=seed)
#vgraph_builder = world.to_drake_plant()
print(len(world.obstacles))


def sample_cfree_handle(n, m, regions=None):
    points = np.zeros((n,3))
    m = m*100
    if regions is None: regions = []		
    for i in range(n):
        bt_tries = 0
        while bt_tries<m:
            point = world.sample_cfree(1)[0]
            #point = world.sample_cfree(1)[0]
            if point_near_regions(point, regions, tries = 5, eps = 0.1):
                bt_tries+=1
            else:
                break
        # if bt_tries == m:
        #     return points, True
        points[i] = point
    return points, False

def estimate_coverage(regions):
    n_s = 5000
    samples = world.sample_cfree(n_s)
    in_s = 0
    for s in samples:
        if point_in_regions(s, regions):
            in_s+=1
    return (1.0*in_s)/n_s

from vislogging import LoggerClique3D
from visibility_clique_decomposition import VisCliqueDecomp
from region_generation import generate_regions_ellipses_multi_threading
from time import strftime,gmtime
from functools import partial


loggerccd = LoggerClique3D(world, f"village_{ap_names[approach]}_{village_side}_{village_height}_"+strftime("%m_%d_%H_%M_%S_", gmtime())+"_", seed, N, 1, eps, estimate_coverage)
def iris_ellipse_w_obstacles_handle(points, ellipsoids, old_regions = None):
    if len(points)>=1:
        #+ region_obstacles
        obstacles = [r for r in world.obstacles]
        regions, _, is_full = generate_regions_ellipses_multi_threading(points, ellipsoids, obstacles, world.iris_domain, estimate_coverage, coverage_threshold=1-eps, old_regs = old_regions, maxiters=1)
    return regions, is_full

VCD = VisCliqueDecomp(  N = N,
                        eps = eps,
                        max_iterations=300,
                        sample_cfree=sample_cfree_handle,
                        col_handle= world.col_handle,
                        build_vgraph= world.vgraph_handle,
                        iris_w_obstacles= iris_ellipse_w_obstacles_handle,
                        verbose=True,
                        logger=loggerccd, 
                        approach = approach,
                    )

VCD.run()

world.plot_regions(VCD.regions)
plot_regions(world.meshcat_drake, VCD.regions)

# Add polytope as a linear constraint via r.A() - np.inf(1), r.b())
# or AddPointInSetConstraints
# HPolytope, ConvexSet is the parent class


In [ ]:
import numpy as np
import networkx as nx
from pydrake.all import MathematicalProgram, Solve, HPolyhedron, VPolytope, Evaluate, eq

In [ ]:
# Run this cell for the dummy case
regions = []
regions.append(HPolyhedron.MakeBox((0, 0), (5, 3))) # Box 0
regions.append(HPolyhedron.MakeBox((3, 2), (8, 7))) # Box 1
regions.append(HPolyhedron.MakeBox((6, 3), (9, 9))) # Box 2
regions.append(HPolyhedron.MakeBox((7, 1), (11, 4))) # Box 3
regions.append(HPolyhedron.MakeBox((10, 3), (14, 9))) # Box 4

In [ ]:
regions = VCD.regions
region_adjacency = {i: {} for i in range(len(regions))}
intersections = {}

G = nx.Graph()

for i, region in enumerate(regions):
    for j in range(i+1, len(regions)):
        if region.IntersectsWith(regions[j]):
            intersection = regions[i].Intersection(regions[j])
            intersections[intersection] = (i, j)
            region_adjacency[i][j] = intersection
            region_adjacency[j][i] = intersection

for intersection in intersections:
    for i, region in enumerate(regions):
        if intersection.IntersectsWith(region):
            for in_i in (region_adjacency[i].values()):
                if (in_i != intersection):
                    G.add_edge(intersection, in_i)

intersection_adjacency = nx.adjacency_matrix(G).todense()
N_intersections = len(intersections.keys())
N_regions = len(regions)
N_dims = regions[0].ambient_dimension()

print(intersection_adjacency)
# keep things clean, as simple as possible, don´t focus on speed yet
# check for redundancy = yes
# And reduce inequalities
# use networkx A star
# use networkx's existing methods to ask it for an adjacency matrix, for A star, put it into this graph form FIRST

In [ ]:
# 1. Define an instance of MathematicalProgram
prog = MathematicalProgram()

# 2. Add decision varaibles
x = prog.NewContinuousVariables(N_intersections, N_dims)

pairwise_dists = []
for i in range(N_intersections-1):
    for j in range(i+1, N_intersections):
        if intersection_adjacency[i][j]:
            pairwise_dists.append(np.linalg.norm(x[i] - x[j]))

# 3. Add Cost function
prog.AddCost(np.sum(np.array(pairwise_dists)))

# # 4. Add Constraints
for i, intersection in enumerate(intersections.keys()):
    intersection.AddPointInSetConstraints(prog, x[i])

# 5. Solve the problem
result = Solve(prog)

# 6. Get the solution
soln = None
if result.is_success:
    soln = result.GetSolution()
    soln = np.asarray(soln, dtype=int).reshape(N_dims, N_intersections).T

soln[1][0] = 6
soln[3][0] = 8
print(soln)

weighted_intersection_adjacency = np.zeros((N_intersections, N_intersections))
weighted_edges = []
intersect = list(intersections.keys())
for i in range(N_intersections-1):
    for j in range(i+1, N_intersections):
        if intersection_adjacency[i][j]:
            weight = soln[i] - soln[j]
            weighted_edges.append((intersect[i], intersect[j], np.linalg.norm(weight)))
            weighted_intersection_adjacency[i][j] = np.linalg.norm(weight)
            weighted_intersection_adjacency[j][i] = np.linalg.norm(weight)

print(weighted_edges)
G.add_weighted_edges_from(weighted_edges)
pos=nx.spring_layout(G) # pos = nx.nx_agraph.graphviz_layout(G)
nx.draw_networkx(G,pos)
labels = nx.get_edge_attributes(G,'weight')
nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)

# add 2 nodes, get shortest path

In [ ]:
start_node = (3, 2, 1)
end_node = (8, 10, 9)

start_region = None
end_region = None
for i in range(len(regions)):
    if regions[i].PointInSet(start_node): start_region = i
    if regions[i].PointInSet(end_node): end_region = i

G.add_node(start_node)
G.add_node(end_node)
for intersection in region_adjacency[start_region].values():
    G.add_edge(start_node, intersection)
for intersection in region_adjacency[end_region].values():
    G.add_edge(end_node, intersection)
    
pos=nx.spring_layout(G) # pos = nx.nx_agraph.graphviz_layout(G)
nx.draw_networkx(G,pos)
labels = nx.get_edge_attributes(G,'weight')
nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)

In [ ]:
path = nx.astar_path(G, start_node, end_node)
print(path)
print(len(path))

In [ ]:
# 1. Define an instance of MathematicalProgram
prog = MathematicalProgram()

# 2. Add decision varaibles
x = prog.NewContinuousVariables(len(path), N_dims)

pairwise_dists = []
for i in range(len(path)-1):
    prog.Add2NormSquaredCost(np.hstack([np.eye(N_dims), -np.eye(N_dims)]), np.zeros(N_dims), x[i:i+2].flatten())
    # pairwise_dists.append(np.linalg.norm(x[i+1] - x[i]))

# # 3. Add Cost function
# prog.AddCost(np.sum(pairwise_dists))

# # 4. Add Constraints
prog.AddConstraint(eq(x[0], path[0]))
prog.AddConstraint(eq(x[-1], path[-1]))
for i in range(1, len(path)-1):
    print(path[i])
    path[i].AddPointInSetConstraints(prog, x[i])

# 5. Solve the problem
result = Solve(prog)

# 6. Get the solution
nodes = None
if result.is_success:
    nodes = result.GetSolution()
    nodes = np.asarray(nodes, dtype=int).reshape(N_dims, len(path)).T

print(nodes)

# inter_1 = list(intersections.keys())[0]
# print(inter_1)
# print(inter_1.PointInSet((5, 1)))

# TODO Potential problem...why is 5, 1 returned when that's not in intersection 1?

In [ ]:
result.get_solver_id().name()

In [ ]:
path_length = len(nodes)
intersections_path = path
final_path = [start_node]
for j in range(1, path_length-1):
    a = nodes[j-1]
    z = nodes[j]
    b = nodes[j+1]

    # Solve minimal distance without adding a box
    # 1. Define an instance of MathematicalProgram
    prog = MathematicalProgram()

    # 2. Add decision variables
    z_nobox = prog.NewContinuousVariables(N_dims)

    # 3. Add Cost function
    prog.AddCost(np.square(np.linalg.norm(z_nobox - a)) + np.square(np.linalg.norm(b - z_nobox)))

    # Add Constraints
    intersections_path[j].AddPointInSetConstraints(prog, z_nobox)

    # 5. Solve the problem
    result = Solve(prog)

    # 6. Get the solution
    if result.is_success:
        node_nobox = result.GetSolution()
        min_nobox = result.get_optimal_cost()

    # Find potential intersections
    box_a, box_b = intersections[intersections_path[j]]
    shared_boxes = set(region_adjacency[box_a].keys()).intersection(set(region_adjacency[box_b].keys()))

    shared_intersections = [(region_adjacency[box_a][i], region_adjacency[box_b][i]) for i in shared_boxes]

    # # Solve minimal distance WITH adding a box
    # # Loop through potential intersections, solve mathematical program for each
    min_box = np.inf
    for box_za, box_zb in shared_intersections:# union intersections[j]
    # 1. Define an instance of MathematicalProgram
        prog = MathematicalProgram()

        # 2. Add decision variables
        z_box = prog.NewContinuousVariables(2, N_dims)
        z_box_1 = z_box[0]
        z_box_2 = z_box[1]

        # 3. Add Cost function
        prog.AddCost(np.square(np.linalg.norm(z_box_1 - a)) + np.square(np.linalg.norm(b - z_box_2)) + np.square(np.linalg.norm(b - z_box_2)))

        # Add Constraints
        box_za.AddPointInSetConstraints(prog, z_box_1)
        box_zb.AddPointInSetConstraints(prog, z_box_2)

        # 5. Solve the problem
        result = Solve(prog)

        # 6. Get the solution
        if result.is_success and result.get_optimal_cost() < min_box:
            nodes_box = np.array(result.GetSolution()).reshape(N_dims, 2).T
            node_box_a = nodes_box[0, :]
            node_box_b = nodes_box[1, :]
            min_box = result.get_optimal_cost()

    if min_box < min_nobox:
        print("add box")
        final_path.append(tuple(node_box_a))
        final_path.append(tuple(node_box_b))
    else:
        print("no add box")
        final_path.append(tuple(node_nobox))
        
final_path.append(end_node)
print(final_path)